<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session6-Assignment/Sentiment_Analysis_using_encoder_decoder(tweetdataset)_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('drive/My Drive/END2/Session5-Assignment/tweets.csv')
df.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [3]:
df.shape

(1364, 2)

In [4]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [5]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 
import torch.nn as nn
import torch.optim as optim


# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [6]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [7]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [8]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [9]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [10]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [11]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [12]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'Admit',
  'women',
  'to',
  'Augusta',
  'golf',
  'club',
  ':',
  'US',
  'President',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://t.co/PVKrepqI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [13]:
Tweet.build_vocab(train,max_size=5000)
Label.build_vocab(train,max_size=5000)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [14]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [16]:
import os, pickle
with open('drive/My Drive/END2/Session6-Assignment/tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

### Defining encoder class

In [175]:
class Encoder(nn.Module):
  def __init__(self,input_dim,emb_dim,hid_dim,n_layers):
    super().__init__()

    self.hid_dim=hid_dim
    self.embedding = nn.Embedding(input_dim, emb_dim) 
    self.rnn= nn.LSTM(emb_dim, hid_dim,  num_layers=n_layers, batch_first=True)

  def forward(self, text,text_lengths):

      embedded =self.embedding(text)

      packaged_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)

      packed_output, (hidden, cell) = self.rnn(packaged_embedded)

      print(f'Output of encoder at every step:{packed_output[0]}') ## output of each word is stroed in packaged output, it keeps appending the hidden vector after every step that is nothing but after every word

      print(f'Output of encoder at last step:{hidden}') ### Hidden contains output of last time step
    
      return hidden,cell



### Defining Decoder class

In [176]:
import torch.nn.functional as F
class Decoder(nn.Module):
  def __init__(self,out_dim,hid_dim,n_layers):
    super().__init__()

    self.hid_dim = hid_dim

    self.out_dim = out_dim

    self.rnn= nn.LSTM(hid_dim,hid_dim, num_layers=n_layers)

    self.fc = nn.Linear(hid_dim,out_dim)


  def forward(self,input,hidden,cell):

    output , (hidden,cell) = self.rnn(hidden,(hidden,cell))

    print(f'Output of decoder at every step (Note that here it is single step):{output[0]}')

    prediction = self.fc(hidden.squeeze(0))

    return prediction,hidden,cell


    


### Combining encoder decoder 

In [177]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        print(encoder.hid_dim)
        print(decoder.hid_dim)
        
        assert encoder.hid_dim == decoder.hid_dim
        "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, text, text_len):

        #hidden, cell = self.encoder(text,text_len)

        hidden, cell = self.encoder(text,text_len)
        
        input = hidden

        output,hidden,cell = self.decoder(input,hidden,cell)

        return output

In [178]:
INPUT_DIM = len(Tweet.vocab)
OUTPUT_DIM = len(Label.vocab)
ENC_EMB_DIM = 256
HID_DIM = 512
NUM_LAYERS = 1

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM,NUM_LAYERS)
dec = Decoder(OUTPUT_DIM,HID_DIM,NUM_LAYERS)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EncoderDecoder(enc, dec, device).to(device)

512
512


In [179]:
model

EncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(4651, 256)
    (rnn): LSTM(256, 512, batch_first=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(512, 512)
    (fc): Linear(in_features=512, out_features=3, bias=True)
  )
)

### Defining loss and accuracy of the model

In [180]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
    
# push to cuda if available
criterion = criterion.to(device)

### Training loop

In [181]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   

  
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  

        # output_dim = predictions.shape[-1]
        
        # predictions = predictions[1:].view(-1, output_dim)
        # batch.labels = batch.labels[1:].view(-1)
        
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        #acc = binary_accuracy(predictions, batch.labels)   
        acc = categorical_accuracy(predictions, batch.labels)   

        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()  

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


### Evaluation Loop

In [182]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets 
            
            #print(batch.labels.shape)

            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()

            # output_dim = predictions.shape[-1]
        
            # predictions = predictions[1:].view(-1, output_dim)
            # batch.labels = batch.labels[1:].view(-1)
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)

            # acc = binary_accuracy(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)   
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Model Training and Evaluation

In [183]:

N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'drive/My Drive/END2/Session6-Assignment/encoder_decoder_classification_saved_weights.pt')
    
    print(f'\t Epoch: {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Epoch: {epoch} | Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Streaming output truncated to the last 5000 lines.
        [-0.1268, -0.0867,  0.0819,  ..., -0.0093, -0.1566, -0.0790],
        [ 0.1261, -0.1523, -0.2224,  ...,  0.0828,  0.0519, -0.1369]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
Output of encoder at last step:tensor([[[-0.0202,  0.0718,  0.0827,  ..., -0.2809,  0.0009, -0.0863],
         [ 0.0925,  0.4877, -0.1284,  ...,  0.3361, -0.1198,  0.0461],
         [ 0.2777,  0.1316,  0.0337,  ..., -0.0521, -0.1474, -0.1820],
         ...,
         [ 0.1864,  0.1036, -0.0098,  ...,  0.0638, -0.1617, -0.0381],
         [-0.1268, -0.0867,  0.0819,  ..., -0.0093, -0.1566, -0.0790],
         [ 0.1261, -0.1523, -0.2224,  ...,  0.0828,  0.0519, -0.1369]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
Output of decoder at every step (Note that here it is single step):tensor([[-0.0198,  0.0804,  0.0372,  ..., -0.1678,  0.0232, -0.1052],
        [ 0.3755,  0.6234, -0.1780,  ...,  0.4969,  0.0479, -0.0357],
        [ 0.1792,  0.093

### Validation of the model by passing the tweets and observing it's outcome along with printing the output from encoder and decoder time steps

In [184]:
path = 'drive/My Drive/END2/Session6-Assignment/encoder_decoder_classification_saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('drive/My Drive/END2/Session6-Assignment/tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)

    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [187]:
twt="Today is a beautiful day"
print(f'Tweet : {twt}')
print(f'Predicted Sentiment : {classify_tweet(twt)} \n')

Tweet : Today is a beautiful day
Output of encoder at every step:tensor([[-0.1027, -0.0503,  0.0479,  ..., -0.0233,  0.0349,  0.0769],
        [-0.0739,  0.1454,  0.0878,  ...,  0.1310,  0.0336,  0.2236],
        [-0.0323,  0.1598,  0.1058,  ...,  0.0845, -0.1329, -0.0104],
        [ 0.0232,  0.1074,  0.0423,  ...,  0.1420,  0.0861, -0.2030],
        [ 0.2048,  0.2209,  0.1606,  ...,  0.2254,  0.1387, -0.0564]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
Output of encoder at last step:tensor([[[ 2.0482e-01,  2.2091e-01,  1.6060e-01,  6.5629e-02,  8.1761e-02,
           2.6118e-01, -1.2571e-01, -1.0286e-02,  1.7628e-01,  1.1670e-01,
          -1.2344e-01,  2.9172e-02, -2.5349e-01, -8.9846e-02, -1.8479e-01,
           1.5307e-01, -3.1001e-01,  2.5397e-02,  1.8019e-02, -4.4491e-02,
          -4.1621e-02,  1.1113e-01,  2.8468e-02,  2.6997e-02,  9.7495e-02,
           9.1819e-02, -2.5758e-02,  8.1477e-02, -1.6929e-01,  6.3044e-02,
          -5.9090e-02, -1.6788e-01,  2.1268e-02,  1

In [186]:
twt="This is my first encoder decoder model"
print(f'Tweet : {twt}')
print(f'Predicted Sentiment : {classify_tweet(twt)} \n')

Tweet : This is my first encoder decoder model
Output of encoder at every step:tensor([[ 1.4525e-01,  6.4722e-05, -4.5118e-04,  ..., -2.3553e-02,
         -9.0755e-02, -9.9952e-02],
        [ 6.5447e-03,  1.6383e-01,  9.3974e-02,  ...,  1.4722e-01,
         -5.0717e-02,  1.2002e-01],
        [-1.4163e-02,  8.2040e-02, -3.9873e-02,  ...,  1.1530e-01,
          8.7571e-02, -3.1883e-02],
        ...,
        [-4.4577e-02,  1.2651e-01,  6.3556e-02,  ...,  1.0086e-01,
          1.6790e-01, -1.0740e-01],
        [-2.3125e-02,  9.3015e-02,  3.5015e-02,  ...,  1.5644e-01,
          2.2985e-01, -2.0054e-01],
        [-1.5929e-03,  5.6669e-02,  2.7312e-02,  ...,  1.7993e-01,
          2.5657e-01, -2.4107e-01]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)
Output of encoder at last step:tensor([[[-1.5929e-03,  5.6669e-02,  2.7312e-02, -1.5404e-03, -1.4514e-01,
           1.8985e-01,  9.3813e-02,  3.0972e-02,  2.9701e-01, -1.0544e-01,
          -2.9301e-01,  1.3024e-02, -2.3836e-01,  9.5386